In [4]:
# Step 1: Install required packages (Uncomment if running on local machine)
# !pip install music21 tensorflow

# Step 2: Import Libraries
import numpy as np
import pickle
from music21 import note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.utils import to_categorical

# Step 3: Dummy note sequence (simulating extracted notes)
notes = ['C4', 'E4', 'G4', 'C5', 'E4', 'G4', 'C5', 'E5', 'G5'] * 10  # Dummy data

# Step 4: Encode notes
note_names = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(note_names))
encoded_notes = [note_to_int[n] for n in notes]

# Step 5: Prepare sequences
sequence_length = 10
network_input = []
network_output = []

for i in range(len(encoded_notes) - sequence_length):
    seq_in = encoded_notes[i:i + sequence_length]
    seq_out = encoded_notes[i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

n_patterns = len(network_input)
n_vocab = len(note_names)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1)) / float(n_vocab)
network_output = to_categorical(network_output)

# Step 6: Build the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Step 7: Train the model (Only 1 epoch for speed)
model.fit(network_input, network_output, epochs=1, batch_size=64)

# Step 8: Save model and encoder
model.save("music_generator_model.h5")
with open("note_encoder.pkl", "wb") as f:
    pickle.dump(note_to_int, f)

print("Training complete. Files 'music_generator_model.h5' and 'note_encoder.pkl' saved.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - loss: 1.7910


Training complete. Files 'music_generator_model.h5' and 'note_encoder.pkl' saved.
